In [1]:
import json
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import shutil
import yaml

In [2]:
# Configuration
TRAIN_LABELS = 'labels_final/imprint_labels_batch_2.json'
PREDICT_CSV = '../data/splits/ml_labelling_batches/ml_labelling_batch_3.csv'
OUTPUT_JSON = 'labels_final/imprint_labels_ml_assisted_batch_3.json'
IMAGE_SOURCE = Path('../data/pillbox_production_images_full_202008')
MODEL_NAME = 'pill_imprint_batch_3'

In [3]:
# Load training labels
with open(TRAIN_LABELS) as f:
    label_data = json.load(f)

# Create character classes and mapping
all_chars = sorted(set(label['label'] for item in label_data for label in item['labels']))
char_to_idx = {char: idx for idx, char in enumerate(all_chars)}

print(f"Classes: {len(all_chars)} characters")
print(f"Training images: {len(label_data)}")

Classes: 62 characters
Training images: 1491


In [4]:
# Setup YOLO dataset structure
base_dir = Path('yolo_dataset')

if base_dir.exists():
    shutil.rmtree(base_dir)
    print("Old dataset removed")

for split in ['train', 'val']:
    (base_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (base_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

# Split data
image_names = [item['image'] for item in label_data]
train_imgs, val_imgs = train_test_split(image_names, test_size=0.15, random_state=42)

print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}")

Old dataset removed
Train: 1267, Val: 224


In [5]:
# Convert to YOLO format and copy images
image_to_labels = {item['image']: item['labels'] for item in label_data}

for img_name, split in [(img, 'train') for img in train_imgs] + [(img, 'val') for img in val_imgs]:
    src = IMAGE_SOURCE / img_name
    if not src.exists():
        continue
    
    # Copy image
    shutil.copy(src, base_dir / split / 'images' / img_name)
    
    # Create YOLO label file
    label_path = base_dir / split / 'labels' / f"{Path(img_name).stem}.txt"
    with open(label_path, 'w') as f:
        for label in image_to_labels[img_name]:
            cls = char_to_idx[label['label']]
            x, y, w, h = map(float, label['coords'].split())
            x, y, w, h = max(0, min(1, x)), max(0, min(1, y)), max(0, min(1, w)), max(0, min(1, h))
            f.write(f"{cls} {x} {y} {w} {h}\n")

print("Dataset prepared")

Dataset prepared


In [6]:
# Create YOLO config file
config = {
    'path': str(base_dir.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(all_chars),
    'names': all_chars
}

with open(base_dir / 'data.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Config created")

Config created


In [7]:
# Train YOLO model
model = YOLO('yolov8n.pt')
results = model.train(
    data=str(base_dir / 'data.yaml'),
    epochs=50,
    imgsz=640,
    batch=16,
    name=MODEL_NAME
)

New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.233 🚀 Python-3.13.2 torch-2.9.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_imprint_batch_3, nbs=64, nms=False, opset=None, optimize=F

In [8]:
# Load CSV and make predictions
predict_df = pd.read_csv(PREDICT_CSV)
trained_model = YOLO(f'../runs/detect/{MODEL_NAME}/weights/best.pt')

predictions = []
for _, row in predict_df.iterrows():
    img_path = IMAGE_SOURCE / row['original_name']
    if not img_path.exists():
        continue
    
    results = trained_model.predict(img_path,
                                    conf=0.15,
                                    agnostic_nms=True,
                                    verbose=False)
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            x, y, w, h = box.xywhn[0].tolist()
            predictions.append({
                'image': row['original_name'],
                'label': all_chars[cls],
                'coords': f"{x} {y} {w} {h}",
                'confidence': conf
            })

print(f"Generated {len(predictions)} predictions")

Generated 5246 predictions


In [9]:
# Group by image and save to JSON
pred_df = pd.DataFrame(predictions)
output_data = []

for img_name in pred_df['image'].unique():
    img_preds = pred_df[pred_df['image'] == img_name]
    output_data.append({
        'image': img_name,
        'labels': [
            {'label': row['label'], 'coords': row['coords'], 'confidence': row['confidence']}
            for _, row in img_preds.iterrows()
        ]
    })

with open(OUTPUT_JSON, 'w') as f:
    json.dump(output_data, f, indent=2)

print(f"\nSaved {len(output_data)} images to {OUTPUT_JSON}")
print(f"\nConfidence stats:")
print(pred_df['confidence'].describe())


Saved 797 images to labels_final/imprint_labels_ml_assisted_batch_3.json

Confidence stats:
count    5246.000000
mean        0.816812
std         0.231926
min         0.150012
25%         0.755113
50%         0.934633
75%         0.970489
max         0.998369
Name: confidence, dtype: float64
